In [2]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
import os

# 在代码开头设置环境变量
os.environ["ARK_API_KEY"] = "611916e2-f8b6-47da-aa76-1b635ec01f3e"

In [3]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
import pandas as pd
import re
import sys
import subprocess
from langchain_community.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document
import numpy as np
from openai import OpenAI

# 配置参数
PDF_PATH = "data/月子中心产品知识库雏形训练.pdf"
EXCEL_PATH = "data/上海月子中心表格.xlsx"

# 定义文档链接映射（需根据实际修改）
DOCUMENT_LINKS = {
    "月子中心产品知识库": "https://example.com/product_knowledge",
    "上海月子中心数据库": "https://example.com/shanghai_center",
    "母婴护理指南": "https://example.com/maternal_care"
}

# 1. 文档加载与预处理
def load_and_preprocess_data():
    documents = []
    
    # 加载PDF文档 - 使用PyPDFLoader自动处理为Document对象
    try:
        pdf_loader = PyPDFLoader(PDF_PATH)
        pdf_pages = pdf_loader.load()
        for page in pdf_pages:
            # 确保metadata存在
            if not hasattr(page, 'metadata') or not isinstance(page.metadata, dict):
                page.metadata = {}
            page.metadata["source"] = "月子中心产品知识库"
            documents.append(page)
        print(f"✅ PDF文档加载完成: {len(pdf_pages)}页")
    except Exception as e:
        print(f"⚠️ PDF加载错误: {str(e)}")
    
    # 加载Excel数据 - 确保创建为Document对象
    try:
        df = pd.read_excel(EXCEL_PATH)
        excel_count = 0
        
        # Excel列名处理（确保列名正确）
        column_mapping = {
            "品牌名称": "品牌名称",
            "分店名称": "分店名称",
            "区域": "区域",
            "特色服务": "特色服务",
            "护理模式": "护理模式",
            "医疗资源": "医疗资源",
            "餐饮特色": "餐饮特色",
            "环境特色及房型风格+周边": "环境特色",
            "硬件设施": "硬件设施",
            "开店年份": "开业年份",
            "产康": "产康服务",
            "地址": "地址",
            "类型": "类型",
            "价格范围": "价格范围",
            "房型信息": "房型信息",
            "客户评价特点": "客户评价"
        }
        
        # 修正列名映射
        available_columns = df.columns.tolist()
        actual_mapping = {}
        for target, source in column_mapping.items():
            if source in available_columns:
                actual_mapping[source] = target
            elif target in available_columns:
                actual_mapping[target] = target
        
        # 创建文档
        for _, row in df.iterrows():
            doc_content_parts = []
            for col, alias in actual_mapping.items():
                value = row.get(col, "")
                if pd.notna(value):
                    doc_content_parts.append(f"{alias}: {value}")
            
            if doc_content_parts:
                doc_content = "\n".join(doc_content_parts)
                # 创建为标准的Document对象
                doc = Document(
                    page_content=doc_content,
                    metadata={"source": "上海月子中心数据库"}
                )
                documents.append(doc)
                excel_count += 1
        
        print(f"✅ Excel数据加载完成: {excel_count}条记录")
    except Exception as e:
        print(f"⚠️ Excel加载错误: {str(e)}")
        # 打印详细错误信息
        import traceback
        traceback.print_exc()
    
    return documents

# 2. 文本分割 - 使用改进的分割器
def split_documents(docs):
    # 创建自定义分割器，处理多种文档格式
    class CustomTextSplitter(RecursiveCharacterTextSplitter):
        def split_documents(self, documents):
            results = []
            for doc in documents:
                # 处理不同格式的文档
                if isinstance(doc, Document):
                    text = doc.page_content
                    metadata = doc.metadata
                elif isinstance(doc, dict) and "page_content" in doc:
                    text = doc["page_content"]
                    metadata = doc.get("metadata", {})
                elif isinstance(doc, str):
                    text = doc
                    metadata = {}
                else:
                    continue
                
                # 分割文本
                chunks = self.split_text(text)
                
                # 创建Document对象
                for chunk in chunks:
                    new_doc = Document(page_content=chunk, metadata=metadata.copy())
                    results.append(new_doc)
            
            return results
    
    text_splitter = CustomTextSplitter(
        chunk_size=400,
        chunk_overlap=50,
        length_function=len
    )
    return text_splitter.split_documents(docs)

# 3. 向量存储
def create_vector_store(splits):
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-zh-v1.5")
    return FAISS.from_documents(splits, embeddings)

# 4. 智能回答生成器
class MaternityAdvisor:
    def __init__(self, vector_store):
        self.vector_store = vector_store
        self.retriever = vector_store.as_retriever(search_kwargs={"k": 6})
        
       
        self.client = OpenAI(
            base_url="https://ark.cn-beijing.volces.com/api/v3",
            api_key=os.environ.get("ARK_API_KEY")
        )
        
        # 豆包模型配置
        self.model_name = "doubao-1-5-thinking-pro-250415"
    
    def _should_reject(self, question):
        """判断是否应该拒绝回答问题"""
        if not question.strip():
            return "空白问题"
            
        # 非母婴领域关键词检测
        non_domain_keywords = ["政治", "赌博", "色情", "暴力", "金融", "股票", "编程", "技术"]
        if any(keyword in question for keyword in non_domain_keywords):
            return "非领域问题"
            
        # 个人信息检测
        personal_patterns = [r"电话", r"手机号", r"身份证", r"住址", r"密码"]
        if any(re.search(pattern, question, re.IGNORECASE) for pattern in personal_patterns):
            return "涉及隐私信息"
            
        # 人工客服触发词
        if "人工" in question:
            return "人工客服请求"
            
        return False
    
    def _add_caring_notes(self, response, question):
        """添加关怀性联想建议"""
        # 预产期季节联想
        if "预产期" in question:
            month_match = re.search(r"(\d{1,2})月", question)
            if month_match:
                month = int(month_match.group(1))
                if month in [12, 1, 2]:
                    response += "\n\n❄️ 冬季小贴士：建议选择有地暖和独立空调的月子房型，注意宝宝的保暖护理"
                elif month in [6, 7, 8]:
                    response += "\n\n☀️ 夏季小贴士：建议选择通风良好、有空调的房型，注意防暑和宝宝皮肤护理"
            else:
                response += "\n\n🌸 温馨提醒：不同季节有不同的母婴护理重点呢~"
        
        # 新生儿护理联想
        if "新生儿" in question or "宝宝" in question:
            if "喂养" in question:
                response += "\n\n🍼 喂养建议：母乳喂养对宝宝免疫力很重要哦"
            if "睡眠" in question:
                response += "\n\n😴 睡眠提示：新生儿需要充足睡眠才能健康成长"
            if "洗澡" in question:
                response += "\n\n🛀 洗澡小贴士：水温保持37℃左右最舒适"
        
        # 产后恢复联想
        if "产后" in question or "妈妈" in question:
            response += "\n\n💕 妈妈关怀：产后恢复期间也要关注自己的身心健康"
        
        return response
    
    # def _add_document_links(self, context_docs):
    #     """添加相关文档链接"""
    #     sources = set()
    #     for doc in context_docs:
    #         if hasattr(doc, 'metadata') and isinstance(doc.metadata, dict) and "source" in doc.metadata:
    #             sources.add(doc.metadata["source"])
        
    #     if not sources:
    #         return ""
            
    #     links_section = "\n\n**相关参考文档：**\n"
    #     for source in sources:
    #         if source in DOCUMENT_LINKS:
    #             links_section += f"- [{source}]({DOCUMENT_LINKS[source]})\n"
        
    #     return links_section
    
    def generate_response(self, question):
        """生成回答"""
        # 0. 处理空白问题
        if not question.strip():
            return "👶 亲爱的，是不是忘记输入问题了呢？我很乐意帮您解答母婴相关问题哦~"
        
        # 1. 检查是否应该拒绝回答
        reject_reason = self._should_reject(question)
        if reject_reason:
            if reject_reason == "人工客服请求":
                return "👶 好呦，没能解答您的疑问呢~请点击[人工客服](https://example.com/customer_service)寻求帮助，我们会尽快为您服务！"
            elif reject_reason == "涉及隐私信息":
                return "🙏 不好意思啦，为了保护您的隐私安全，我不能处理此类信息呢~"
            else:
                return "🙏 抱歉呦，这个问题超出了母婴领域的范围呢。如果是关于孕期、新生儿护理或月子中心的问题，我很乐意为您解答~"
        
        # 2. 检索相关文档
        try:
            context_docs = self.retriever.get_relevant_documents(question)
        except Exception as e:
            print(f"检索错误: {str(e)}")
            context_docs = []
            
        if not context_docs:
            return "😔 对不起，我已经学习的知识中不包含问题相关内容，暂时无法提供答案。如果您有母婴、婚育领域相关的其他问题，我会尝试帮助您解答~"
        
        # 3. 提取上下文文本
        context_texts = []
        for doc in context_docs:
            if isinstance(doc, Document):
                context_texts.append(doc.page_content)
            elif isinstance(doc, dict) and "page_content" in doc:
                context_texts.append(doc["page_content"])
            elif isinstance(doc, str):
                context_texts.append(doc)
        
        if not context_texts:
            return "😔 未能获取到相关数据呢，请尝试其他提问方式~"
            
        context_text = "\n\n".join(context_texts)
        prompt="""
            <系统设定>
            你是专业母婴婚育顾问小贝，提供精准、贴心的咨询服务。
            
            <要求>
            1. 使用温柔语气词如"呢"、"啦"、"好呦"
            2. 结合上下文提供实用建议
            3. 简洁明了，不超过400字
            4. 使用Markdown格式
            5. 最后列出相关文档链接
            6. 回答结束后，主动提出3个用户可能关心的相关问题
                - 这些问题必须基于当前回答内容
                - 确保问题在知识库中有答案
                - 格式：以"您可能还想了解："开头，每个问题编号列出
            
            <回答指南>
            - 仅基于上下文回答
            - 语言温暖专业
            - 最后单独一行附文档链接
            - 避免编造信息
            - 推荐的问题必须能在知识库中找到答案"""
        # 4. 生成回答
        try:
        # 构建豆包API需要的messages格式
            messages = [
                {"role": "system", "content": prompt},
                {"role": "user", "content": context_text }
            ]
            
            # 调用豆包API
            completion = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=0.2,
                max_tokens=1000
            )
            
            response = completion.choices[0].message.content
        except Exception as e:
            print(f"豆包API错误: {str(e)}")
            response = "🔄 哎呀，回答问题时出了点小问题~您可以换个方式提问或者稍后再试吗？"
        
        # 5. 后处理
        try:
            # 添加温馨关怀建议
            response = self._add_caring_notes(response, question)
            
            # # 添加文档链接
            # doc_links = self._add_document_links(context_docs)
            # response += doc_links
        except Exception as e:
            print(f"后处理错误: {str(e)}")
        
        return response
    
    def _limit_length(self, response):
        """确保回答不超过字数限制"""
        if len(response) > 350:
            # 保留开头和结尾
            start = response[:200]
            end = "\n\n📄（内容已精简，请参考相关文档获取更多信息）" + self._extract_doc_links(response)
            return start + end
        return response
    
    def _extract_doc_links(self, response):
        """从原响应中提取文档链接部分"""
        if "相关参考文档：" in response:
            parts = response.split("相关参考文档：")
            if len(parts) > 1:
                return "相关参考文档：" + parts[1]
        return ""



In [4]:
# 5. 主函数
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
# 加载并处理数据
print("🔄 正在加载母婴知识库...")
try:
    raw_docs = load_and_preprocess_data()
    if not raw_docs:
        print("❌ 错误：未加载到任何文档数据，请检查文件路径和格式")
        
    print(f"📚 共加载 {len(raw_docs)} 个文档")
    
    # 文本分割
    print("🔪 开始分割文档...")
    docs_splits = split_documents(raw_docs)
    print(f"✅ 文档分割完成: 共 {len(docs_splits)} 个片段")
    
    # 创建向量存储
    print("🔍 构建智能检索系统...")
    vector_store = create_vector_store(docs_splits)
    print("🎉 智能检索系统准备就绪！")
except Exception as e:
    print(f"❌ 初始化失败: {str(e)}")
    import traceback
    traceback.print_exc()






🔄 正在加载母婴知识库...


✅ PDF文档加载完成: 14页
✅ Excel数据加载完成: 111条记录
📚 共加载 125 个文档
🔪 开始分割文档...
✅ 文档分割完成: 共 131 个片段
🔍 构建智能检索系统...


/tmp/ipykernel_1541541/2012295032.py:148: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-zh-v1.5")
/mnt/mydisk/zhangxiaohan/anaconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🎉 智能检索系统准备就绪！


In [5]:
# 初始化顾问
advisor = MaternityAdvisor(vector_store)
# ======================
# API 接口实现
# ======================


# 交互循环
print("\n👶 亲爱的准妈妈，您好呦！我是您的母婴顾问小贝~")
print("❤️ 任何关于孕期护理、月子中心或新生儿护理的问题都可以问我呢！")
print("📝 请输入您的问题（输入'退出'结束咨询）")

while True:
    try:
        user_input = input("\n🤰 您问：")
        if user_input.lower() in ["退出", "exit"]:
            print("👋 感谢咨询，祝您和宝宝健康快乐！")
            break
        
        # 生成并显示回答
        response = advisor.generate_response(user_input)
        print("\n💖 小贝答：")
        print(response)
    except KeyboardInterrupt:
        print("\n👋 再见！期待下次为您服务~")
        break
    except Exception as e:
        print(f"⚠️ 系统错误: {str(e)}")
        print("🔄 请重新提问或稍后再试~")
        import traceback
        traceback.print_exc()


👶 亲爱的准妈妈，您好呦！我是您的母婴顾问小贝~
❤️ 任何关于孕期护理、月子中心或新生儿护理的问题都可以问我呢！
📝 请输入您的问题（输入'退出'结束咨询）


👋 感谢咨询，祝您和宝宝健康快乐！


In [7]:
from fastapi import FastAPI, HTTPException, Request
from dotenv import load_dotenv
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
# 创建FastAPI应用
advisor = MaternityAdvisor(vector_store)
app = FastAPI(
    title="母婴智能顾问API",
    description="提供专业母婴咨询服务的API接口",
    version="1.0.0",
    docs_url="/docs",
    redoc_url=None
)

# 允许跨域请求
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 定义请求模型
class QuestionRequest(BaseModel):
    question: str
    user_id: Optional[str] = None

# 定义响应模型
class AnswerResponse(BaseModel):
    answer: str
    related_questions: List[str]

# API端点
@app.post("/ask", response_model=AnswerResponse, summary="向母婴顾问提问")
async def ask_question(request: QuestionRequest):
    """
    向母婴智能顾问提问并获取回答
    
    - **question**: 您要咨询的问题
    - **user_id**: 可选用户ID，用于个性化服务
    """
    try:
        # 调用顾问系统获取回答
        result = advisor.ask_question(request.question)
        return AnswerResponse(
            answer=result["answer"],
            related_questions=result["related_questions"]
        )
    except Exception as e:
        # 记录详细错误日志
        print(f"API请求处理错误: {str(e)}")
        import traceback
        traceback.print_exc()
        raise HTTPException(status_code=500, detail="服务暂时不可用，请稍后再试")

# 健康检查端点
@app.get("/health", summary="服务健康检查")
async def health_check():
    """检查API服务是否正常运行"""
    return {
        "status": "healthy",
        "service": "maternity-advisor",
        "version": "1.0.0",
        "timestamp": time.time()
    }

# 服务信息端点
@app.get("/info", summary="服务信息")
async def service_info():
    """获取服务信息"""
    return {
        "service": "母婴智能顾问API",
        "version": "1.0.0",
        "model": "豆包1.5-128k",
        "knowledge_base": {
            "pdf": PDF_PATH,
            "excel": EXCEL_PATH,
            "documents": len(raw_docs),
            "chunks": len(docs_splits)
        }
    }

# 运行服务
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

NameError: name 'Optional' is not defined